# Laptop Pricing Analysis

This notebook explores the pricing of laptops using data cleaning, feature engineering, and visualization.

Steps:
- Load and clean data
- Feature extraction
- Data visualization
- Correlation analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os